In [27]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter('ignore')

In [29]:
data_dir = "F:/TARGET frontier/TXT33/"
sample_data = pd.DataFrame([])
for i in range(2015,2018+1):
    data_name = "train"+str(i)
    res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
    sample_data = pd.concat([sample_data,res])
    
#リークしないように慎重に！
data_name = "predict2019"
res = pd.read_csv(data_dir+data_name+".csv",encoding="shift-jis")#,na_values=['--']
predict_len = len(res)
sample_data = pd.concat([sample_data,res])

tail = "fe8"

In [30]:
pd.set_option('display.max_columns', 100)

,レースID(新),場所,芝・ダ,距離,コース区分,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,多頭出し,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,前走着差タイム,前走PCI3,前走単勝オッズ
0,201512270605081112,中山,芝,1200,A,1000万,良,16,Ｍ．デム,6,牡,3,13,56,492,-4.0,6.0,5.0,ハーツクライ,ロイヤルチャージャー系,NaN,(栗),須貝尚介,1,1,-0.0,5.4,1.9,2.7,190.0,201511140505031003,東京,芝,1400.0,1000万,稍,15.0,*,横山典弘,2.0,先行,54,37.2,33.9,10.0,58.2,59.7,*,8.0,8.0,0.2,59.53,14.6
1,201512270605081111,中山,芝,1200,A,1000万,良,16,ルメール,6,牝,3,10,54,466,-12.0,3.0,3.0,シンボリクリスエス,ロイヤルチャージャー系,NaN,(栗),音無秀孝,2,2,0.0,2.8,1.3,1.6,130.0,201512060704020613,中京,芝,1200.0,500万,良,18.0,NaN,ルメール,7.0,先行,54,34.5,34.6,3.0,46.9,49.7,*,1.0,1.0,-0.4,48.80,2.2
2,201512270605081106,中山,芝,1200,A,1000万,良,16,岩田康誠,3,牡,3,9,56,478,6.0,5.0,2.0,フレンチデピュティ,ニアークティック系,NaN,(美),武市康男,3,3,0.2,6.2,1.8,2.5,180.0,201511210805051217,京都,芝,1200.0,1000万,良,18.0,*,和田竜二,8.0,中団,56,34.5,34.4,9.0,48.0,50.3,*,7.0,7.0,0.4,50.13,28.6
3,201512270605081108,中山,芝,1200,A,1000万,良,16,石川裕紀,4,牝,5,17,55,484,-4.0,2.0,4.0,Fastnet Rock,その他のエクリプス系,NaN,(美),高橋文雅,4,4,0.3,6.6,1.7,2.5,NaN,201512130605040911,中山,芝,1200.0,1000万,稍,16.0,NaN,石川裕紀,6.0,中団,53,34.1,34.4,3.0,46.3,49.1,*,3.0,3.0,0.0,48.17,19.1
4,201512270605081116,中山,芝,1200,A,1000万,良,16,西田雄一,8,牝,3,15,54,440,-6.0,2.0,4.0,アドマイヤコジーン,ナスルーラ系,NaN,(美),星野忍,5,5,0.3,33.9,6.3,9.7,NaN,201512130605040904,中山,芝,1200.0,1000万,稍,16.0,*,嶋田純次,2.0,逃げ,53,33.6,36.4,15.0,46.3,42.3,NaN,13.0,13.0,1.5,48.17,8.6


In [4]:
drop_cols = ["コース区分"]
train_data = sample_data.drop(drop_cols,axis=1)
train_data.shape

(222682, 40)

In [5]:
class Feature_engineering():
    def __init__(self,df):
        self.df = df
        self.l = len(df)
        
    def labelencoding(self,col):
        le = LabelEncoder()
        le.fit(self.df[col])
        self.df[col] = le.transform(self.df[col])
    
    def more_than_double_le(self,lst):
        a,b = lst[0],lst[1]
        le = LabelEncoder()
        le.fit(pd.concat([self.df[a],self.df[b]]))
        self.df[a] = le.transform(self.df[a])
        self.df[b] = le.transform(self.df[b])
        
    def handle_nan(self,col):
        self.df.fillna({col:"--"},inplace=True)
        
    def fill_nan(self,col):
        self.df.fillna({col:"0"},inplace=True)
    
    def handle_mark(self,col):
        self.df[col] = self.df[col].str.replace("△","")
        self.df[col] = self.df[col].str.replace("▲","")
        self.df[col] = self.df[col].str.replace("☆","")
        #["△","▲","☆"]
    
    def handle_class(self,col):
        dic_class = {"500万":"1勝","1000万":"2勝","1600万":"3勝","OP(L)":"ｵｰﾌﾟﾝ","重賞":"Ｇ３","(不明)":"--"}
        self.df[col].replace(dic_class,inplace=True)
        
    def handle_specific_nan(self,col):
        dic_specific = {"----":np.nan}
        self.df[col].replace(dic_specific,inplace=True)
    
    def drop_row(self,col):
        self.df.dropna(inplace=True,subset = [col])
        self.df.reset_index(drop=True,inplace=True)
    
    def handle_abs(self,col):
        self.df[col] = self.df[col].astype("float64")
        self.df[col] = self.df[col].where(self.df[col]>0 , 0) #0より小さいなら0を入れてやる
    
    def set_validation(self,y_col,ozz_col):
        self.df[ozz_col] = self.df[ozz_col].astype("float64")
        self.l = len(self.df)
        res = np.array([0.0]*self.l)
        
        for i in range(self.l):
            res[i] = (1/math.exp(3*self.df[y_col][i]))*(1/math.exp(1/self.df[ozz_col][i]*5)) #2/math.exp(1/i*3) min(50,self.df[ozz_col][i]) #1/math.exp(3*self.df[y_col][i])
        self.df[y_col] = np.round(res,5)
    
    #わざと着差をいじる
    def essence(self,y_col,ozz_col):
        res = []
        for i,j in zip(self.df[y_col],self.df[ozz_col]):
            if i == 0 and 9 <= j <= 13:
                res.append(i-0.3)
            else:
                res.append(i)
        self.df[y_col] = pd.DataFrame(res)
    
    def ranking_and_number(self,ID_col):
        ranking = []
        number = []
        num = 0
        rank = 1
        res = ""
        for i,j in enumerate(self.df[ID_col]):
            j = str(j)
            if i == 0:
                ranking.append(rank)
                rank += 1
                res = j[:-2]
                num += 1
                continue
            if res != j[:-2]:
                res = j[:-2]
                number += [num]*num
                rank = 1
                ranking.append(rank)
                rank += 1
                num = 1
            else:
                num += 1
                ranking.append(rank)
                rank += 1
        number += [num]*num
        print(len(number))
        print(len(ranking))
        self.df = pd.concat([self.df,pd.Series(ranking,name="順位"),pd.Series(number,name="頭数")],axis=1)
        
    def basyo_kyori_sibadart(self,basyo,kyori,sibadart):
        self.l = len(self.df)
        res = []
        for i in range(self.l):
            b = self.df[basyo][i]
            k = self.df[kyori][i]
            s = self.df[sibadart][i]
            if any(j == np.nan for j in [b,k,s]):
                res.append(np.nan)
            else:
                res.append(str(b)+str(k)+str(s))
        le = LabelEncoder()
        le.fit(res)
        res = le.transform(res)
        name = basyo+"距離芝ダ"
        res = pd.Series(res,name=name)
        self.df =  pd.concat([self.df,res],axis=1)
                
        


In [6]:
fe = Feature_engineering(train_data)

In [7]:
nan_cols = ["父タイプ名","前走場所","前芝・ダ","前クラス名","前走馬場状態","前走脚質","前好走"]
for i in nan_cols:
    print("processing:",i)
    fe.handle_nan(i) #'--'埋めする

processing: 父タイプ名
processing: 前走場所
processing: 前芝・ダ
processing: 前クラス名
processing: 前走馬場状態
processing: 前走脚質
processing: 前好走


In [8]:
fill_nan_with_0 = ["複勝配当","休み明け〜戦目"]
for i in fill_nan_with_0:
    print("processing:",i)
    fe.fill_nan(i)

processing: 複勝配当


In [9]:
mark_cols = ["斤量","前走斤量"]
for i in mark_cols:
    print("processing:",i)
    fe.handle_mark(i)

processing: 斤量
processing: 前走斤量


In [10]:
handle_cls = ["クラス名","前クラス名"]
for i in handle_cls:
    print("processing:",i)
    fe.handle_class(i)

processing: クラス名
processing: 前クラス名


In [11]:
double_le = [["場所","前走場所"],["芝・ダ","前芝・ダ"],["馬場状態","前走馬場状態"],["クラス名","前クラス名"]]
for i in double_le:
    print("processing:",i)
    fe.more_than_double_le(i)

processing: ['場所', '前走場所']
processing: ['芝・ダ', '前芝・ダ']
processing: ['馬場状態', '前走馬場状態']
processing: ['クラス名', '前クラス名']


In [12]:
cat_cols = ["性別","種牡馬","父タイプ名","前走脚質","前好走"]
for i in cat_cols:
    print("processing:",i)
    fe.labelencoding(i)

processing: 性別
processing: 種牡馬
processing: 父タイプ名
processing: 前走脚質
processing: 前好走


In [13]:
specific_nan = ["前走着差タイム","着差"]
for i in specific_nan:
    print("processing",i)
    fe.handle_specific_nan(i)

processing 前走着差タイム
processing 着差


In [14]:
drop_na = ["単勝オッズ"]
for i in drop_na:
    print("processing",i)
    fe.drop_row(i)

processing 単勝オッズ


In [15]:
handle_abs = ["前走着差タイム","着差"]
for i in handle_abs:
    print("processing:",i)
    fe.handle_abs(i)

processing: 前走着差タイム
processing: 着差


In [16]:
"""essence = [["着差","単勝オッズ"]]
for i,j in essence:
    print("processing:",i,j)
    fe.essence(i,j)"""

'essence = [["着差","単勝オッズ"]]\nfor i,j in essence:\n    print("processing:",i,j)\n    fe.essence(i,j)'

In [17]:
fe.ranking_and_number("レースID(旧)")
train_data = fe.df #これでranking_and_numberが反映される

222063
222063


In [18]:
bks = [["場所","距離","芝・ダ"],["前走場所","前距離","前芝・ダ"]]
for b,k,s in bks:
    print("processing:",b,k,s)
    fe.basyo_kyori_sibadart(b,k,s)
    train_data = fe.df

processing: 場所 距離 芝・ダ
processing: 前走場所 前距離 前芝・ダ


In [19]:
train_data.head()

,場所,芝・ダ,距離,クラス名,馬場状態,騎手コード,枠番,性別,年齢,キャリア,斤量,間隔,休み明け〜戦目,種牡馬,父タイプ名,調教師コード,前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走騎手コード,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走PCI3,前走RPCI,前PCI,前好走,前走平均1Fタイム,前走着差タイム,前走単勝オッズ,着差,レースID(旧),複勝オッズ下限,複勝オッズ上限,複勝配当,単勝オッズ,順位,頭数,場所距離芝ダ,前走場所距離芝ダ
0,2,2,1200,2,3,5212,6,2,3,13,56,6.0,5.0,572,8,1105,13,2,1400.0,2,2,660.0,2.0,3,54,37.2,33.9,59.53,58.2,59.7,0,11.93,0.2,14.6,0.0,0615581112,1.9,2.7,190,5.4,1,16,63,42
1,2,2,1200,2,3,5339,6,1,3,10,54,3.0,3.0,452,8,1002,1,2,1200.0,1,3,5339.0,7.0,3,54,34.5,34.6,48.80,46.9,49.7,0,11.52,0.0,2.2,0.0,0615581111,1.3,1.6,130,2.8,2,16,63,15
2,2,2,1200,2,3,5203,3,2,3,9,56,5.0,2.0,607,5,1089,3,2,1200.0,2,3,1018.0,8.0,2,56,34.5,34.4,50.13,48.0,50.3,0,11.48,0.4,28.6,0.2,0615581106,1.8,2.5,180,6.2,3,16,63,188
3,2,2,1200,2,3,1150,4,1,5,17,55,2.0,4.0,91,1,1134,2,2,1200.0,2,2,1150.0,6.0,2,53,34.1,34.4,48.17,46.3,49.1,0,11.42,0.0,19.1,0.3,0615581108,1.7,2.5,0,6.6,4,16,63,111
4,2,2,1200,2,3,1004,8,1,3,15,54,2.0,4.0,325,4,1030,2,2,1200.0,2,2,1134.0,2.0,5,53,33.6,36.4,48.17,46.3,42.3,1,11.67,1.5,8.6,0.3,0615581116,6.3,9.7,0,33.9,5,16,63,111


In [20]:
all_len = len(train_data)
all_len

222063

In [21]:
predict_len

40422

In [22]:
train = train_data.iloc[:all_len-predict_len-1]

In [23]:
predict = train_data.iloc[all_len-predict_len:]

In [24]:
len(predict)

40422

In [25]:
train.to_csv(data_dir+"train"+tail+".csv",index=False,encoding="shift-jis")
predict.to_csv(data_dir+"predict"+tail+".csv",index=False,encoding="shift-jis")

In [26]:
predict.dtypes

場所             int32
芝・ダ            int32
距離             int64
クラス名           int32
馬場状態           int32
騎手コード          int64
枠番             int64
性別             int32
年齢             int64
キャリア           int64
斤量            object
間隔           float64
休み明け〜戦目      float64
種牡馬            int32
父タイプ名          int32
調教師コード         int64
前走場所           int32
前芝・ダ           int32
前距離          float64
前クラス名          int32
前走馬場状態         int32
前走騎手コード      float64
前走枠番         float64
前走脚質           int32
前走斤量          object
前走Ave-3F     float64
前走上り3F       float64
前走PCI3       float64
前走RPCI       float64
前PCI         float64
前好走            int32
前走平均1Fタイム    float64
前走着差タイム      float64
前走単勝オッズ      float64
着差           float64
レースID(旧)      object
複勝オッズ下限      float64
複勝オッズ上限      float64
複勝配当          object
単勝オッズ        float64
順位             int64
頭数             int64
場所距離芝ダ         int64
前走場所距離芝ダ       int64
dtype: object